### Consumindo api spotify
* Isntalando api spotify

**Extraindo base de dados para analise exploratoria** 
  * Spotify

In [1]:
pip install spotipy

### Espaço para importações de bibliotecas utilizadas

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import sys
from requests import get
import time

### Espaço para autenticação de usuário para acesso a API **spotify**

In [3]:
#Autenticação api
cid ="75b0d6f3334a4b689620eda9f917844a" 
secret = "ca50b30849d24b5d88508296ee9e6d98" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, 
client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False

#### **Funções usadas nesse notebook**

In [4]:
def get_album_data(album_id):
    time.sleep(gera_um_numero_segundos())
    try: 

       data = sp.album(album_id)
       return data
    except:
       return None  

In [5]:
def get_album_data_artista(album_id):
    try:
      data = sp.album(album_id)
      return data['artists']
    except:
      return None    

In [6]:
def  get_uri(artista_data):
    time.sleep(gera_um_numero_segundos())
    try:
       return artista_data[0]['uri']
    except:
       return None  

In [7]:
def get_artist(artista_id):
  time.sleep(gera_um_numero_segundos())
  try:
    resultado = sp.artist(artista_id)
    return resultado
  except:
    return None  

In [8]:
def get_album_imagem(album_dados):
   time.sleep(gera_um_numero_segundos())
   try:
       img = album_dados['images'][1]['url']
       return img
   except:
       return None   

In [9]:
def get_url_img(artista_data):
    time.sleep(gera_um_numero_segundos())
    try:
      url = artista_data['images'][1]['url']
      return url
    except:
      return None  

In [10]:
def get_artist_genres(artista_data):
    time.sleep(gera_um_numero_segundos())
    try:
      genre = artista_data['genres'] 
      return genre
    except:
      return None    

In [11]:
def aplica_genero_album(x):
  
  try:
      dados_album = get_album_data(x)
      time.sleep(gera_um_numero_segundos())
      dados_artista_album = get_album_data_artista(x)
      uri_artista = get_uri(dados_artista_album)
      time.sleep(gera_um_numero_segundos())
      dado_artista = get_artist(uri_artista)
      time.sleep(gera_um_numero_segundos())
      genero_artista = get_artist_genres(dado_artista)
      return genero_artista 
  except:
      return None   

In [12]:
def get_id_artista(nome):
  time.sleep(gera_um_numero_segundos())
  resul = sp.search(q=nome,type = 'artist')
  try:
      items = resul['artists']['items']
      nome_busca = items[0]['name']
      if len(items) > 0:
        if nome == nome_busca:
          return items[0]['id']
      else:
        return None 
  except:
      return None  

In [13]:
def aplica_img_album(x):
  try:
      dados_album = get_album_data(x)
      time.sleep(gera_um_numero_segundos())
      url_img = get_album_imagem(dados_album)
      return url_img
  except:
      return None    

In [14]:
def aplica_genero_artist(nome):
    
    time.sleep(gera_um_numero_segundos())

    try:
        nome = nome.strip()
        id_artista = get_id_artista(nome.strip())
        time.sleep(gera_um_numero_segundos())
        artita = get_artist(id_artista)
        time.sleep(gera_um_numero_segundos())
        genero = get_artist_genres(artita)
        return genero
    except:
        return None   


In [15]:
def gera_um_numero_segundos():
     return np.random.randint(4,8)
   

### Lendo arquivo csv streaming spotify

In [16]:
path_file_csv = '/content/drive/MyDrive/Projeto-Final/Data_Mateus/spotify/in_csv/streamingspotify.csv'

In [17]:
#Leitura csv para dataframe
dfStreaming = pd.read_csv(path_file_csv)

In [18]:
dfStreaming.head()

,endTime,artistName,trackName,msPlayed,endDate,endMonth
0,04:57:00,Padre Fábio De Melo,Não Desisto,3.945083,2020-07-24,jul
1,05:01:00,Padre Fábio De Melo,Proteção,4.012883,2020-07-24,jul
2,05:06:00,Padre Fábio De Melo,Iluminar,4.876983,2020-07-24,jul
3,05:12:00,Guilherme Sá,Oração de São Francisco - Bonus Track,5.559333,2020-07-24,jul
4,05:15:00,Padre Fábio De Melo,Trem-Bala,3.405333,2020-07-24,jul


In [ ]:
dfStreaming['genres'] = dfStreaming['artistName'].apply(aplica_genero_artist)

In [ ]:
dfStreaming.to_csv('/content/drive/MyDrive/Projeto-Final/Data_Mateus/spotify/in_csv/streamingspotify.csv',index = False)

**Lendo CSV para df dos albums MateusSpotify**

In [ ]:
path_file_csv = '/content/drive/MyDrive/Projeto-Final/Data_Mateus/spotify/in_csv/albumsSpotifyMateus.csv'

In [ ]:
dfAlbumsSpotify = pd.read_csv(path_file_csv)

In [ ]:
dfAlbumsSpotify.head(30)

In [ ]:
dfAlbumsSpotify.info()

In [ ]:
#Aplicando genero a alguns albums pelo artista 
dfAlbumsSpotify['genre'] = dfAlbumsSpotify['uri'].apply(aplica_genero_album)

In [ ]:
dfAlbumsSpotify.head()

In [ ]:
#Aplicando ao artista que estão com nome varios artista o genero pela sua descrição.
dfAlbumsSpotify.loc[2]['genre'] = 'forró'
dfAlbumsSpotify.loc[3]['genre'] = 'funk'

In [ ]:
#Dividindo  a coluna genero, ja que é uma lista de genero.
dfAlbumsSpotify = dfAlbumsSpotify.explode('genre',ignore_index= True)
dfAlbumsSpotify.head(10)

In [ ]:
#Aplicando uma nova coluna ao df com a informação de link da imagem do album
dfAlbumsSpotify['imageAlbum'] = dfAlbumsSpotify.query('artist != "Various Artists"')['uri'].apply(aplica_img_album)